### Semester Project

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import os

# Load the video file
video_path = 'footage.mp4'  # Replace with your actual video path

# Check if the video file exists
if not os.path.isfile(video_path):
    print("Error: The video file does not exist.")
    exit()

cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

# Create an object detector using KLT (Kanade-Lucas-Tomasi) algorithm
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the first frame
ret, old_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    cv2.destroyAllWindows()
    exit()

# Convert the first frame to grayscale
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

# Detect corners in the first frame
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

# Generate random colors for tracking points
color = np.random.randint(0, 255, (100, 3))

# Directory to save frames
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

frame_idx = 0

while True:
    # Read a new frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks and bounding boxes
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)
    
    # Calculate the bounding box for each good point
    for i, new in enumerate(good_new):
        x, y = new.ravel()
        cv2.rectangle(frame, (int(x)-15, int(y)-15), (int(x)+15, (int(y)+15)), (255, 0, 0), 2)
    
    img = cv2.add(frame, mask)

    # Save the frame as an image
    frame_filename = os.path.join(output_dir, f'frame_{frame_idx:04d}.png')
    cv2.imwrite(frame_filename, img)
    
    # Display the frame in the notebook
    plt.figure(figsize=(10, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    display(plt.gcf())
    clear_output(wait=True)
    plt.close()

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)
    
    frame_idx += 1

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

print(f"Frames extracted and saved in {output_dir}")


Frames extracted and saved in output
